# Tutorial: Cognitive Service Vision Planogram Compliance using Python

This is a tutorial about using Cognitive Service Vision Planogram Compliance.

Cognitive Service Vision Planogram Compliance are tools to check planogram compliance using planogram and product recognition result. This allows users to compare the actual arrangement of products on the shelves with the predetermined planogram, identifying any deviations and taking necessary corrective actions. The combination of accurate product recognition and retail execution KPIs empowers retailers to enhance their store layouts, improve customer experiences, and optimize sales performance.

Currently, planogram compliance feature are available in **EastUS**, **West US2**, and **West Europe** regions.

Please create a computer vision resource on Azure portal, in **EastUS**, **West US2**, or **West Europe** region, if you don't already have one. You can use [Multi-service resource](https://learn.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Canomaly-detector%2Clanguage-service%2Ccomputer-vision%2Cwindows) as well. 

![check ../model_customization/media/create_cv_resource.png if pic does not show up](../model_customization/media/create_cv_resource.png)

## Install the python samples package

Install the sample code including utility code helping you use Python to run planogram compliance in this tutorial.

In [ ]:
pip install cognitive-service-vision-model-customization-python-samples

In [ ]:
import json
import logging
import uuid
logging.getLogger().setLevel(logging.INFO)

import cv2
import numpy as np
from PIL import Image

from cognitive_service_vision_model_customization_python_samples import ResourceType
from cognitive_service_vision_model_customization_python_samples.clients import PlanogramComplianceClient, ProductRecognitionClient
from cognitive_service_vision_model_customization_python_samples.models import PlanogramMatchingRequest, ProductRecognition
from cognitive_service_vision_model_customization_python_samples.tools import visualize_matching_result, visualize_planogram

## Credentials

Resource name and resource key are needed for accessing the service, which you can find here:

![check media/credentials.png if pic does not show up](./resources/credentials.png)

If you are using a multi-service resource, endpoint can be found here:

![check resources/multi_service_endpoint.png if pic does not show up](./resources/multi_service_endpoint.png)

In [ ]:
# Resource and key
resource_type = ResourceType.SINGLE_SERVICE_RESOURCE # or ResourceType.MULTI_SERVICE_RESOURCE

resource_name = None
multi_service_endpoint = None

if resource_type == ResourceType.SINGLE_SERVICE_RESOURCE:
    resource_name = '{specify_your_resource_name}'
    assert resource_name
else:
    multi_service_endpoint = '{specify_your_service_endpoint}'
    assert multi_service_endpoint

resource_key = '{specify_your_resource_key}'

## Prepare and load the planogram

A planogram, also known as a plano, POG, plan-o-gram, schematic, or visual description, is a detailed visual map that establishes the position of the merchandise within a retail store. It is often used in the retail industry as a tool for space planning and product placement. This diagram shows precisely where each item in a store should be placed in order to maximize customer purchases.

Below is a visualization of a sample planogram

In [ ]:
planogram = json.load(open('./resources/sample_planogram.json', 'r'))
viz_planogram = visualize_planogram(planogram, 800, './resources/sample_catalogs')
viz_planogram = cv2.cvtColor(viz_planogram, cv2.COLOR_BGR2RGB)
display(Image.fromarray(viz_planogram))

## Call prebuilt product recognition model

In [ ]:
client = ProductRecognitionClient(resource_type, resource_name, multi_service_endpoint, resource_key)
run_name = str(uuid.uuid4())
model_name = 'ms-pretrained-product-detection'
run = ProductRecognition(run_name, model_name)

with open('./resources/sample_image.jpg', 'rb') as f:
    img = f.read()

try:
    client.create_run(run, img, 'image/png')
    recognition_result = client.wait_for_completion(run_name, model_name)
finally:
    client.delete_run(run_name, model_name)

## Call planogram matching (With prebuilt product recognition result)

Planograme compliance will be most useful if you have trained a customized model that can recognize detailed products, so the planogram matching can help you identify if a product is placed in the correct place.

With only the prebuilt product recognition model result, it's still useful to to planogram matching as it helps you identify which products on the shelf are out of stock and needs restock.

In [ ]:
client = PlanogramComplianceClient(resource_type, resource_name, multi_service_endpoint, resource_key)
matching_request = PlanogramMatchingRequest(recognition_result.result, planogram)
matching_result = client.match_planogram(matching_request)

## Visualize matching result (With prebuilt product recognition result)

In the below result, the green boxes are planogram positions that are compliant, meaning that there's a product is placed at the right position on the shelf, the red boxes indicate that the position is not compliant because no product is detected so it's a gap.

In [ ]:
# Convert img data to numpy array and decode numpy array to OpenCV BGR image
cv_img = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)
cv_img = visualize_matching_result(cv_img, matching_result.to_dict(), planogram)
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
display(Image.fromarray(cv_img))

## Call planogram matching (With customized model product recognition result)

Here we provid a sample response from a customized model, where detailed product names are recognized, we use this to match with the planogram using the planogram matching API

In [ ]:
customized_recognition_result = json.load(open('./resources/sample_customized_recognition_result.json', 'r'))
client = PlanogramComplianceClient(resource_type, resource_name, multi_service_endpoint, resource_key)
matching_request = PlanogramMatchingRequest(customized_recognition_result, planogram)
matching_result = client.match_planogram(matching_request)

## Visualize matching result (With prebuilt product recognition result)

In the resulting image, color-coded boxes are used to depict the compliance status of various planogram positions. The green boxes represent positions that are fully compliant, denoting a perfect correlation between the detected product and the intended planogram configuration. On the other hand, the magenta boxes emphasize instances of non-compliance, where a product has been detected, but it does not correspond to the product assigned in the planogram. Finally, the red boxes point out non-compliant positions due to gaps, where no product is detected where one should exist.

In [ ]:
# Convert img data to numpy array and decode numpy array to OpenCV BGR image
cv_img = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)
cv_img = visualize_matching_result(cv_img, matching_result.to_dict(), planogram)
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
display(Image.fromarray(cv_img))

## Compare it with the planogram

Comparing the above realogram with the below planogram to validate the correctness of matching

In [ ]:
display(Image.fromarray(viz_planogram))